# Compute variances and means from cityscape dataset


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.apis.inference import LoadImage
from mmseg.core.evaluation import get_palette
from mmseg.datasets.pipelines import Compose
from mmseg.ops import resize

import mmcv
from mmcv.parallel import collate, scatter
from torchvision import io
import torch
import torch.nn.functional as F
from torchvision import transforms as T
import numpy as np
import glob

/home/cariad/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/usr/local/lib/python3.8/dist-packages/torchvision-0.14.0a0+a67cc87-py3.8-linux-x86_64.egg/torchvision/image.so


In [4]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [5]:
config = mmcv.Config.fromfile('../packages/mmsegmentation/configs/segformer/segformer_mit-b5_8x1_1024x1024_160k_cityscapes.py')
checkpoint_file = 'checkpoints/mmseg/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth'

In [6]:
model = init_segmentor(config, checkpoint_file, device='cuda:0')

/home/cariad/packages/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: checkpoints/mmseg/segformer_mit-b5_8x1_1024x1024_160k_cityscapes_20211206_072934-87a052ec.pth


In [7]:
IMAGES_ROOT = '/home/datasets/cityscapes/leftImg8bit/train/'

In [8]:
images = glob.glob(IMAGES_ROOT+'*/*.png')
images[:10]

['/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000073_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000102_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000076_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000133_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000104_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000018_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000006_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000098_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000009_000019_leftImg8bit.png',
 '/home/datasets/cityscapes/leftImg8bit/train/tubingen/tubingen_000083_000019_leftImg8bit.png']

In [ ]:
def load_image2(path):
    image_data = mmcv.imread(path)
    return {
        'filename': path,
        'ori_filename': path,
        'img': image_data,
        'img_shape': image_data.shape,
        'ori_shape': image_data.shape
    }
    
pipeline_img = Compose([load_image2] + config.data.test.pipeline[1:])
label_img = Compose([load_image2])

pred_list = None

for i, image in enumerate(images):
    label = torch.from_numpy(label_img(image.replace('leftImg8bit','gtFine').replace('.png','_labelTrainIds.png'))['img'][1::4, 1::4, 0]).cuda()
    data = pipeline_img(image)
    with torch.no_grad():
        x = model.extract_feat(torch.stack(data['img']).cuda())
        out = model._decode_head_forward_test(x, [])
        # add train label to predictions, and max-logits (value and index)
        max_logits =  out.max(1)
        outputs =torch.cat((out, label[None, None, :, :], max_logits[0][None, :, :], max_logits[1][None, :, :]), axis=1)
        
        if pred_list is None:
            pred_list = outputs.data.cpu()
        else:
            pred_list = torch.cat((pred_list, outputs.cpu()), 0)
        del outputs
        del out

    # calc partial result
    if i % 50 == 49 or i == len(images) - 1:
        print(f'processed {i} of {len(images)}')
        v_ = []
        mean_dict, var_dict, med_dict, min_dict, max_dict = {}, {}, {}, {}, {}
        for cls in range(19):
            label_filter = (pred_list[:, 19] == cls)
            n_c = label_filter.sum()
            x_c = pred_list[:, cls][label_filter]
            m_c = x_c.mean() # 1 / n_c * torch.sum(x_c) # mean
            v_c = x_c.var() # 1 / n_c * torch.sum((x_c - m_c))**2 # variance
            
            mean_dict[cls] = m_c.item()
            var_dict[cls] = v_c.item()
            if len(x_c) > 0:
                min_dict[cls] = x_c.min().item()
                max_dict[cls] = x_c.max().item()
                med_dict[cls] = x_c.median().item()
                
        #print(f"class mean: {mean_dict}")
        #print(f"class var: {var_dict}")
        np.save(f'stats/cityscape_{i+1}_mean.npy', mean_dict)
        np.save(f'stats/cityscape_{i+1}_var.npy', var_dict)
        np.save(f'stats/cityscape_{i+1}_min.npy', min_dict)
        np.save(f'stats/cityscape_{i+1}_max.npy', max_dict)
        np.save(f'stats/cityscape_{i+1}_median.npy', med_dict)
            
            #v_.append(torch.sum(torch.matmul(x_c - m_c, torch.t(x_c - m_c))))
        # shared covariance
        #s = 1/20 *torch.sum(torch.sum(
        #v_ = torch.stack(v_)

        


processed 49 of 2975
class mean: {0: 6.655925273895264, 1: 4.0771484375, 2: 2.305957794189453, 3: 3.374669313430786, 4: 4.148864269256592, 5: 4.154590129852295, 6: 3.6148645877838135, 7: 5.890624046325684, 8: 4.34588098526001, 9: 4.777425289154053, 10: 4.359645843505859, 11: 5.249518394470215, 12: 4.224935054779053, 13: 4.225487232208252, 14: 3.332580804824829, 15: 4.105919361114502, 16: nan, 17: 3.7792327404022217, 18: 4.182849884033203}
class var: {0: 1.8402049541473389, 1: 1.5704984664916992, 2: 1.991491436958313, 3: 1.5749142169952393, 4: 3.0364913940429688, 5: 4.931636810302734, 6: 2.305152654647827, 7: 7.149972915649414, 8: 1.5168023109436035, 9: 2.6290335655212402, 10: 1.345596194267273, 11: 5.040283203125, 12: 4.654160976409912, 13: 2.55873703956604, 14: 1.7930071353912354, 15: 2.0828161239624023, 16: nan, 17: 3.2799153327941895, 18: 4.346231937408447}
processed 99 of 2975
class mean: {0: 6.655076503753662, 1: 4.098357200622559, 2: 2.3850207328796387, 3: 3.519454002380371, 4: 3

In [ ]:
np.save(f'stats/cityscape_mean.npy', mean_dict)
np.save(f'stats/cityscape_var.npy', var_dict)

In [ ]:
c

In [ ]:
m_c.min()

In [ ]:
raise RuntimeError

In [ ]:
label_filter.shape

In [ ]:
import h5py
from pathlib import Path

DIR_CHECKPOINTS = Path("/home/datasets/checkpoints/")


In [ ]:
estimates_path = DIR_CHECKPOINTS / "cityscapes_train_estimates_global.h5"
with h5py.File(estimates_path, "r") as data:
    arithmetic_means = np.array(data['means'])
    #self.arithmetic_means = self.arithmetic_means.reshape(self.arithmetic_means.shape[:2] + (-1,))
    inverse_covariances = np.array(data['inverse'])
    #inverse_covariances = torch.from_numpy(self.inverse_covariances.astype("float"))

In [ ]:
display(arithmetic_means.shape)
arithmetic_means

In [ ]:
display(inverse_covariances.shape)
inverse_covariances

In [ ]:
inverse_covariances[0,0]

In [ ]:
inverse_covariances[1,0]

In [ ]:
torch.cuda.empty_cache()    

In [11]:
m1000 = np.load(f'stats/cityscape_1000_mean.npy', allow_pickle=True)
m1000

array({0: 6.486597061157227, 1: 4.01138162612915, 2: 2.4171581268310547, 3: 3.362257719039917, 4: 3.737880229949951, 5: 3.8902792930603027, 6: 3.769230365753174, 7: 5.21175479888916, 8: 4.336178302764893, 9: 4.404292106628418, 10: 4.525447845458984, 11: 4.731545448303223, 12: 4.460194110870361, 13: 4.112583160400391, 14: 4.182897567749023, 15: 3.978729724884033, 16: 4.060572147369385, 17: 4.191195487976074, 18: 4.130975246429443},
      dtype=object)

In [14]:
m2250 = np.load(f'stats_500/cityscape_mean.npy', allow_pickle=True)
m2250

array({0: 6.580508232116699, 1: 3.7664437294006348, 2: 2.5598835945129395, 3: nan, 4: 2.555851697921753, 5: 4.376359462738037, 6: 2.704991579055786, 7: 6.755075931549072, 8: 4.22168493270874, 9: 3.3676230907440186, 10: 4.716228485107422, 11: 2.8417561054229736, 12: 3.528679370880127, 13: 4.085790157318115, 14: 2.452819585800171, 15: nan, 16: nan, 17: 4.525382995605469, 18: 3.720585584640503},
      dtype=object)